**Dataset 1 (Pos), 10 rows per participant for each MW/ PC probe**

In [1]:
import pandas as pd

def begaze_to_df(file_path, pnum, condition): #txt file, participant number, condition
    
    '''
    All BeGaze eye data into a data frame.
    '''
    
    # create dataframe
    columns = ['Event Type', 'Trial', 'Number', 'Start', 'End/ Event', 'Duration/ Description(Events)', 'F:LocX / S:StartLocX', 'F:LocY / S:StartLocY', 
               'F:DispX / S:EndLocX', 'F:DispY / S:EndLocY', 'F:AOIhit / S:Amplitude', 'F:Plane / S:PeakSpeed', 'F:AvPupSizeX / S:PeakSpeedAt', 
               'F:AvPupSizeY / S:AverageSpeed', 'S:PeekAccel', 'S:PeekDecel', 'S:AverAccel']

    df = pd.DataFrame(columns=columns)

    with open(file_path, 'r') as file:
        
        # skip first 22 lines
        for _ in range(21):
                next(file)
        
        for line in file:
            new_line = line.split('\t')            

            # Check if the number of elements in the line matches the number of columns (excluding 'P' and 'Condition')
            if len(new_line) < len(columns): 
                new_line.extend(['NA'] * (len(columns) - len(new_line))) # Pad the new_line with 'NA' to match the number of columns

            df.loc[len(df.index)] = new_line # Append the line to the DataFrame
    
    df.insert(0, 'P', pnum) # add participant number to df
    df.insert(1, 'Condition', condition) # add condition to df

    return df

In [2]:
def fix_MW_PC(df):
    
    '''
    Get a subset of the df with all right-eye fixations, MW events and PC events.
    '''
    
    # get Fixation R
    fixations = df['Event Type'] == 'Fixation R'
    # get MW events
    MW_event = df['End/ Event'].str.contains(r'# Message: MW\d+\.rtf', regex=True)
    # get PC events
    PC_event = df['End/ Event'].str.contains(r'# Message: PC\d+\.rtf', regex=True)
    # get page start trigger events, excluding the first one
    pagestart_event = df['End/ Event'].str.contains(r'# Message: Trigger\d+\.rtf', regex=True) & ~df['End/ Event'].str.contains(r'# Message: Trigger00\.rtf')

    # combine filters, logical OR
    combined_filter = fixations | MW_event | PC_event | pagestart_event
    subset_df = df[combined_filter]

    return subset_df

In [3]:
import re

def MW_PC_response(df):
    
    '''
    Get responses to MW and PC probes. If response wasn't recorded = NA and manually added later.
    '''
    
    columns = ['Participant','ProbeNum', 'MW', 'PC']
    df_probes = pd.DataFrame(columns=columns)
    
    # get participant number from df
    pnum = df['P'].iloc[0]
    
    # initlalize number of probe to 1
    probenum = 1
    
    for idx, row in df.iterrows():
    
        mw = None
        pc = None
        
        # get MW answers
        match_mw = re.match(r'# Message: PC(\d+)\.rtf', row['End/ Event'])
        
        if match_mw:
            pc_value = match_mw.group(1)
            
            # get last 3 fixations from MW
            previous_row_1_mw = df.at[idx-1, 'F:AOIhit / S:Amplitude']
            previous_row_2_mw = df.at[idx-2, 'F:AOIhit / S:Amplitude']
            previous_row_3_mw = df.at[idx-3, 'F:AOIhit / S:Amplitude']  
            
            # if any of the last fixations is a trigger, change mw variable accordingly
            if "TriggerAOI001" in [previous_row_1_mw, previous_row_2_mw, previous_row_3_mw]: # no MW
                mw = 0
                pc = "NA"
                
            elif "TriggerAOI002" in [previous_row_1_mw, previous_row_2_mw, previous_row_3_mw]: # MW
                mw = 1
                pc = None
                
            else:
                mw = "NA" # no answer recorded
                pc = "NA"
                
            # in case of MW, get PC answers
            if mw == 1:
                for idx, row in df.iterrows():
                    match_pc = re.match(r'# Message: Trigger%s.rtf' % pc_value, row['End/ Event'])

                    if match_pc:
                        previous_row_1_pc = df.at[idx-1, 'F:AOIhit / S:Amplitude']
                        previous_row_2_pc = df.at[idx-2, 'F:AOIhit / S:Amplitude']
                        previous_row_3_pc = df.at[idx-3, 'F:AOIhit / S:Amplitude']

                        # if any of the last fixations is a trigger, change pc variable accordingly
                        if "TriggerAOI001" in [previous_row_1_pc, previous_row_2_pc, previous_row_3_pc]: # PC made
                            pc = 1
                        elif "TriggerAOI002" in [previous_row_1_pc, previous_row_2_pc, previous_row_3_pc]: # PC not made
                            pc = 0
                        else:
                            pc = "NA"
                    # else:
                    #     pc = "NA" # no PC event matched the MW probe
            
            insertList = [pnum, probenum, mw, pc]
            df_probes.loc[len(df_probes)] = insertList
            
            probenum += 1

    return df_probes

In [4]:
def fix_20spriorMW(df1, df2, condition):
    
    '''
    Extract fixations which occur 20 seconds preceding the MW probe.
    '''
    
    columns = ['Condition', 'ProbeStart', 'ClosestFix', 'ActualDif', 'NumFix', 'FixDur']
    df_fix = pd.DataFrame(columns=columns)
    
    # get dict of MW probe time
    probe_times = {}
    for idx, row in df1.iterrows():
        if re.match(r'# Message: MW\d+\.rtf', row['End/ Event']):
            mw_number = re.match(r'# Message: MW(\d+)\.rtf', row['End/ Event']).group(1) # extract MW number
            probe_times["MW" + mw_number] = row['Start']
    
    mw_probe_count = 0
    condition_neu = 'neu'
            
    # get fixation approx. 20s before probe
    for probe in probe_times: # iterate through probes
        
        probe_start = probe_times[probe] # probe start time
        
        # calculate probe_start - 20s in microseconds
        twenty_sec_in_micro = 20000000
        twenty_sec_prior_probe = int(probe_start) - twenty_sec_in_micro
        
        # get all fixation start times
        fixstart_all = []
        for fixstart in df1["Start"]:
            if int(fixstart) <= int(probe_start):
                fixstart_all.append(int(fixstart))
        
        # get fixation closest to twenty_sec_prior_probe
        closest_fix = min(fixstart_all, key=lambda x: abs(x - twenty_sec_prior_probe))
        
        # get actual time between probe_start and closest_fix in seconds
        actual_dif = round( ((int(probe_start) - closest_fix) / 1000000), 1 )
    
        # get all fixations 20s preceding the probe
        fix_20s_num = []
        for fixnum in df1["Start"]:
            if int(fixnum) < int(probe_start) and int(fixnum) >= int(closest_fix):
                fix_20s_num.append(fixnum)
        
        # number of fixations
        numfix = len(fix_20s_num)
        
        # duration of fixations
        fix_20s_dur = []
        for idx, row in df1.iterrows():
            if row["Start"] in fix_20s_num:
                try:
                    fix_20s_dur.append(int(row["Duration/ Description(Events)"]))
                except ValueError:
                    continue
        
        fixdur = int(sum(fix_20s_dur) / len(fix_20s_dur))
        
        if mw_probe_count < 5:
            insertList = [condition_neu, probe_start, closest_fix, actual_dif, numfix, fixdur]
            df_fix.loc[len(df_fix.index)] = insertList
            mw_probe_count += 1
        else:
            insertList = [condition, probe_start, closest_fix, actual_dif, numfix, fixdur]
            df_fix.loc[len(df_fix.index)] = insertList
            
    # concatenate with participant num, probenum, mw, pc
    extended_df = pd.concat([df2, df_fix], axis=1)
    
    return extended_df

In [ ]:
P_list = ['10','12','14','16','18','20','22','24','26','30','3','32','34','36','38','40']

for i in P_list:
    # all data to df
    P_all = begaze_to_df('P_P'+i+'.txt', pnum='P'+i, condition='pos')
    
    # subset of fixations, MW, PC, Triggers
    P_all_subset = fix_MW_PC(P_all)
    P_all_subset.reset_index(drop=True, inplace=True) #reset indices
    
    # get MW, PC responses
    P_MW_PC = MW_PC_response(P_all_subset)
    
    # final df
    P_final = fix_20spriorMW(P_all_subset, P_MW_PC, condition='pos')
    
    # convert to csv
    P_final.to_csv('P_P'+i+'_final.csv', encoding='utf-8', index=False)